In [ ]:
# ============================================
# 1) IMPORT LIBRARY
# ============================================
import pandas as pd
import random
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
import torch
from datasets import Dataset


import torch
from torch import nn

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ==================================================
# 2) BACA DATA
# ==================================================
df = pd.read_csv("/content/cleaned_for_training_final (3).csv")

print("\n=== Step 2: Dataset Loaded ===")
print("Jumlah total data :", len(df))
print("\nDistribusi Label (SEBELUM SPLIT):")
print(df['label'].value_counts())
print("\nPersentase Label (SEBELUM SPLIT):")
print((df['label'].value_counts(normalize=True) * 100).round(4))


In [ ]:
from sklearn.model_selection import train_test_split

# ==================================================
# 3) SPLIT DATA (70:15:15) DENGAN STRATIFY
# ==================================================
train_df, test_df = train_test_split(
    df,
    test_size=0.15,
    random_state=42,
    stratify=df["label"]
)

train_df, valid_df = train_test_split(
    train_df,
    test_size=0.1765,
    random_state=42,
    stratify=train_df["label"]
)

print("Train:", len(train_df), "Valid:", len(valid_df), "Test:", len(test_df))
print("Distribusi label train sebelum PROS:")
print(train_df["label"].value_counts(), "\n")


In [ ]:
from collections import Counter
import pandas as pd

def apply_PROS(df, label="label", target_ratio=1.5, seed=42):
    counter = df[label].value_counts()
    minority = counter.idxmin()
    majority = counter.idxmax()

    df_min = df[df[label] == minority]
    df_maj = df[df[label] == majority]

    target_min_size = int(len(df_maj) / target_ratio)

    if target_min_size <= len(df_min):
        return df.sample(frac=1, random_state=seed).reset_index(drop=True)

    df_min_over = df_min.sample(n=target_min_size, replace=True, random_state=seed)
    return pd.concat([df_maj, df_min_over]).sample(frac=1, random_state=seed).reset_index(drop=True)


# ==================================================
# PROS DIAPLIKASIKAN KE TRAIN
# Catatan:
# - target_ratio -> (1.5:1)
# 
# ==================================================

train_pros = apply_PROS(train_df, label="label", target_ratio=1.5, seed=42)
print(train_pros["label"].value_counts().sort_index())


print("Distribusi label TRAIN sebelum PROS:")
print(train_df["label"].value_counts().sort_index(), "\n")

print("Distribusi label TRAIN sesudah PROS:")
print(train_pros["label"].value_counts().sort_index(), "\n")

print("Persentase sesudah PROS:")
print((train_pros["label"].value_counts(normalize=True).sort_index() * 100).round(2))


In [ ]:
# ==================================================
# 6) CLASS WEIGHT BERDASARKAN TRAIN_PROS
# ==================================================
labels_pros = train_pros["label"]
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels_pros),
    y=labels_pros
)

# override manual
class_weights[0] = 1.0
class_weights[1] = 1.2

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

print(class_weights_tensor)

In [ ]:
# ============================================
# 6) TOKENIZER
# ============================================
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "indolem/indobertweet-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

model.to(device)


In [ ]:
def tokenize_function(batch):
    return tokenizer(
        batch["clean_text"],          # pastikan nama kolom ini ada di CSV
        padding="max_length",
        truncation=True,
        max_length=128
    )

In [ ]:
from datasets import Dataset

# ==================================================
# 8) CONVERT PANDAS → HF DATASET
# ==================================================
# TRAIN (PROS)
train_ds = Dataset.from_pandas(train_pros.reset_index(drop=True))
train_ds = train_ds.map(tokenize_function, batched=True)
train_ds = train_ds.remove_columns(
    [c for c in train_ds.column_names if c not in ["input_ids", "attention_mask", "label"]]
)
train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# VALID
valid_ds = Dataset.from_pandas(valid_df.reset_index(drop=True))
valid_ds = valid_ds.map(tokenize_function, batched=True)
valid_ds = valid_ds.remove_columns(
    [c for c in valid_ds.column_names if c not in ["input_ids", "attention_mask", "label"]]
)
valid_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# TEST
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))
test_ds = test_ds.map(tokenize_function, batched=True)
test_ds = test_ds.remove_columns(
    [c for c in test_ds.column_names if c not in ["input_ids", "attention_mask", "label"]]
)
test_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])



In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, preds)
    precision, recall, f1_macro, _ = precision_recall_fscore_support(
        labels, preds,
        average="macro",
        zero_division=0
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1_macro": f1_macro,
    }

In [ ]:
# ============================================
# 8) WEIGHT TENSOR & WEIGHTED TRAINER
# ============================================
from transformers import Trainer
import torch.nn as nn
import torch

class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=0):
        labels = inputs.get("labels")

        # Hapus labels sebelum masuk ke model
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        )

        logits = outputs.get("logits")

        # CrossEntropyLoss dengan class weight
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits, labels)

        # FIX return
        if return_outputs:
            return loss, outputs
        else:
            return loss


In [ ]:
import itertools
import random
import torch

# Ruang hyperparameter
search_space = {
    "learning_rate": [1e-5, 2e-5, 3e-5, 5e-5],
    "batch_size": [8, 16, 32],
    "num_train_epochs": [3, 4, 5]
}

# Buat semua kombinasi (lr, bs, ep) → anti duplikasi
all_combos = list(itertools.product(
    search_space["learning_rate"],
    search_space["batch_size"],
    search_space["num_train_epochs"]
))

random.shuffle(all_combos)   # acak urutan kombinasi

N_TRIALS = 8   # jumlah percobaan yang mau dicoba
all_combos = all_combos[:N_TRIALS]   # ambil N_TRIALS kombinasi pertama



In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, EarlyStoppingCallback
import random

model_name = "indolem/indobertweet-base-uncased"
best_f1 = 0.0
best_params = None
results = []

# misal all_combos sudah kamu buat dan dibatasi N_TRIALS
# all_combos = [...]
# all_combos = all_combos[:N_TRIALS]

for i, (lr, bs, ep) in enumerate(all_combos, start=1):
    print(f"\n==================== Percobaan {i} ====================")
    print(f"lr={lr}, bs={bs}, epoch={ep}")

    # reset model tiap trial
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
    ).to(device)

    args = TrainingArguments(
        output_dir=f"./tmp_indobertweet_prus_classweight_trial_{i}",
        learning_rate=lr,
        per_device_train_batch_size=bs,
        per_device_eval_batch_size=bs,
        num_train_epochs=ep,
        eval_strategy="epoch",      # ✅ perbaiki ini
        logging_strategy="epoch",
        save_strategy="no",
        load_best_model_at_end=False,
        metric_for_best_model="f1_macro",
        greater_is_better=True,
        report_to="none",
        seed=42
    )

    trainer = WeightedTrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        class_weights=class_weights_tensor,   # ✅ nama argumen sesuai __init__
        callbacks=[
            EarlyStoppingCallback(
                early_stopping_patience=2,
                early_stopping_threshold=0.0
            )
        ]
    )

    trainer.train()

    # ambil log per epoch
    history = trainer.state.log_history
    f1_per_epoch = [h["eval_f1_macro"] for h in history if "eval_f1_macro" in h]
    best_f1_trial = max(f1_per_epoch)
    best_epoch_idx = f1_per_epoch.index(best_f1_trial) + 1

    results.append({
        "trial": i,
        "learning_rate": lr,
        "batch_size": bs,
        "num_train_epochs": ep,
        "best_epoch": best_epoch_idx,
        "f1_macro": best_f1_trial
    })

    print(f"➡️ F1 Macro terbaik percobaan {i}: {best_f1_trial:.4f} (epoch {best_epoch_idx})")

    if best_f1_trial > best_f1:
        best_f1 = best_f1_trial
        best_params = {
            "learning_rate": lr,
            "batch_size": bs,
            "num_train_epochs": ep,
            "best_epoch": best_epoch_idx
        }

print("\n==================== Hasil Random Search (PRUS + Class Weight) ====================")
print("BEST:", best_params)
print("BEST F1:", best_f1)

In [ ]:
import pandas as pd

log_hist = pd.DataFrame(trainer.state.log_history)


In [ ]:
import matplotlib.pyplot as plt

# Filter for training logs (where 'loss' is present)
train_logs = log_hist[log_hist["loss"].notna()]
# Filter for evaluation logs (where 'eval_loss' is present)
eval_logs = log_hist[log_hist["eval_loss"].notna()]

plt.plot(train_logs["epoch"], train_logs["loss"], label="Train Loss")
plt.plot(eval_logs["epoch"], eval_logs["eval_loss"], label="Eval Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train vs Eval Loss")
plt.legend()
plt.grid(True)
plt.show()

Manual

In [ ]:
best_params = {'learning_rate': 2e-05, 'batch_size': 16}
print("Defined best_params:", best_params)

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    EarlyStoppingCallback,
)

# ambil hyperparameter terbaik dari random search
best_lr     = best_params["learning_rate"]
best_bs     = best_params["batch_size"]

print("Best params:", best_params)

# model baru untuk training final
model_final = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
).to(device)

args_final = TrainingArguments(
    output_dir="./indobertweet_prus_classweight_best",
    learning_rate=best_lr,
    per_device_train_batch_size=best_bs,
    per_device_eval_batch_size=best_bs,
    num_train_epochs=3,          # train sampai epoch optimal
    eval_strategy="epoch",          # ✅ diperbaiki
    logging_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    weight_decay=0.01,                    # regularisasi biar ga makin overfit
    report_to="none",
    seed=42,
)

trainer_final = WeightedTrainer(
    model=model_final,
    args=args_final,
    train_dataset=train_ds,               # pastikan ini sudah PRUS
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    class_weights=class_weights_tensor,          # untuk class weight
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=2    # stop cepat kalau mulai overfit
        )
    ]
)

trainer_final.train()

# simpan model & tokenizer buat dipakai lagi / lampiran skripsi
save_dir = "./indobertweet_prus_classweight_best"
trainer_final.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

print("Model final disimpan di:", save_dir)


In [ ]:
import pandas as pd

# evaluasi
eval_valid = trainer_final.evaluate(eval_dataset=valid_ds)
eval_test  = trainer_final.evaluate(eval_dataset=test_ds)

# pilih metrik yang mau ditampilkan
keep_keys = [
    "eval_loss",
    "eval_accuracy",
    "eval_precision",
    "eval_recall",
    "eval_f1_macro",
    "epoch"
]

def filter_metrics(dct, name):
    data = {"Dataset": name}
    for k in keep_keys:
        data[k] = dct.get(k, None)
    return data

rows = [
    filter_metrics(eval_valid, "Valid"),
    filter_metrics(eval_test,  "Test")
]

df_eval = pd.DataFrame(rows)

# urutkan kolom + rename biar lebih enak dibaca
df_eval = df_eval[[
    "Dataset",
    "eval_loss",
    "eval_accuracy",
    "eval_precision",
    "eval_recall",
    "eval_f1_macro",
    "epoch"
]].rename(columns={
    "eval_loss": "Loss",
    "eval_accuracy": "Accuracy",
    "eval_precision": "Precision",
    "eval_recall": "Recall",
    "eval_f1_macro": "F1 Macro",
    "epoch": "Epoch"
})

# bulatkan 4 desimal untuk metrik numerik
for col in ["Loss", "Accuracy", "Precision", "Recall", "F1 Macro"]:
    df_eval[col] = df_eval[col].astype(float).round(4)

df_eval = df_eval.set_index("Dataset")
df_eval


In [ ]:
import pandas as pd

log_hist = pd.DataFrame(trainer_final.state.log_history)


In [ ]:
import matplotlib.pyplot as plt

# Filter for training logs (where 'loss' is present)
train_logs = log_hist[log_hist["loss"].notna()]
# Filter for evaluation logs (where 'eval_loss' is present)
eval_logs = log_hist[log_hist["eval_loss"].notna()]

plt.plot(train_logs["epoch"], train_logs["loss"], label="Train Loss")
plt.plot(eval_logs["epoch"], eval_logs["eval_loss"], label="Eval Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train vs Eval Loss")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
eval_f1 = log_hist[log_hist["eval_f1_macro"].notna()]

plt.plot(eval_f1["epoch"], eval_f1["eval_f1_macro"], marker='o')
plt.xlabel("Epoch")
plt.ylabel("F1 Macro")
plt.title("Validation F1 per Epoch")
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

pred_test = trainer_final.predict(test_ds)
y_pred = np.argmax(pred_test.predictions, axis=1)
y_true = pred_test.label_ids

# Print classification report
print("\n📊 Classification Report (TEST):")
print(classification_report(y_true, y_pred, target_names=["non-sarkasme", "sarkasme"], digits=4))

# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["non-sarkasme", "sarkasme"],
            yticklabels=["non-sarkasme", "sarkasme"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix (TEST)")
plt.show()

In [ ]:
save_dir = "./model_pros_final"

trainer_final.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

print("Model & tokenizer disimpan ke:", save_dir)


Visualisasi

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

pred_test = trainer_final.predict(test_ds)
logits = pred_test.predictions
y_true = pred_test.label_ids

probs = np.exp(logits) / np.exp(logits).sum(axis=1, keepdims=True)
y_score = probs[:, 1]

fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

print(f"ROC–AUC Score: {roc_auc:.4f}")

plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.4f}", linewidth=2)
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Random Guess")
plt.title("ROC Curve – Deteksi Sarkasme")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(df_thresh["threshold"], df_thresh["f1_sarkas"], marker="o")
plt.title("F1 Score Kelas Sarkasme vs Threshold")
plt.xlabel("Threshold")
plt.ylabel("F1 Score (Sarkasme)")
plt.grid(alpha=0.3)
plt.show()


In [ ]:
sentences = [
    "Bagus banget nih! Luar biasa! aku juga mau #kaburajadulu ke thailand",
    "pemerintah udah #kaburajadulu masalah negara, kita kapan?",
    "kapan ya bisa #kaburajadulu",
    "Pemerintah selalu bikin rakyat sengasara #kaburajadulu"]

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
outputs = model_final(**inputs)
preds = outputs.logits.argmax(dim=1)

for s, p in zip(sentences, preds):
    print(s, "=>", "Sarkasme" if p.item()==1 else "Non-Sarkasme")


BATAS